In [2]:
import itertools
from tree_from_assoc_rules import list_all_factors
from controller import generate_decision_tree
from tabulate import tabulate
import datetime

In [3]:
# Define all factors - Ref(3)
all_factors = ["stay", "length_of_stay", "country", "review_group", "room_type"]

# Read file of possible values
possible_factor_values = list_all_factors()

In [4]:
# All possible permutations of all factors - Ref(4)
all_factors_combinations = []
for length in range(2, len(all_factors) + 1):
    all_factors_combinations += list(itertools.combinations(all_factors, length))

print(all_factors_combinations)

[('stay', 'length_of_stay'), ('stay', 'country'), ('stay', 'review_group'), ('stay', 'room_type'), ('length_of_stay', 'country'), ('length_of_stay', 'review_group'), ('length_of_stay', 'room_type'), ('country', 'review_group'), ('country', 'room_type'), ('review_group', 'room_type'), ('stay', 'length_of_stay', 'country'), ('stay', 'length_of_stay', 'review_group'), ('stay', 'length_of_stay', 'room_type'), ('stay', 'country', 'review_group'), ('stay', 'country', 'room_type'), ('stay', 'review_group', 'room_type'), ('length_of_stay', 'country', 'review_group'), ('length_of_stay', 'country', 'room_type'), ('length_of_stay', 'review_group', 'room_type'), ('country', 'review_group', 'room_type'), ('stay', 'length_of_stay', 'country', 'review_group'), ('stay', 'length_of_stay', 'country', 'room_type'), ('stay', 'length_of_stay', 'review_group', 'room_type'), ('stay', 'country', 'review_group', 'room_type'), ('length_of_stay', 'country', 'review_group', 'room_type'), ('stay', 'length_of_stay'

In [5]:
# Hardcoded some constants
mcda_type = "rating"
hotel_name = "Amari Phuket"
rule_year = None
rule_month = None

In [6]:
print("all cases = " + str(len(all_factors_combinations)))
def collect_paths(node, route_nodes=[]):
    result = []
    has_child = len(node.children) > 0
    if has_child:
        for child_node in node.children:
            new_route_nodes = []
            new_route_nodes += route_nodes
            new_route_nodes.append(node)
            result += collect_paths(child_node, route_nodes=new_route_nodes)
    else:
        new_route_nodes = []
        new_route_nodes += route_nodes
        new_route_nodes.append(node)
        return [new_route_nodes]
    return result

# Generate decision tree for all permutations
count = 0
gen_count = 0
nodes = []
result_score_table = [["Outcome", "Factor_1", "Factor_2", "Factor_3", "Factor_4", "Factor_5", "Score"]]
for idx, permutation in enumerate(all_factors_combinations):
    # Calculate weights array
    factors_length = len(permutation)
    factor_weight = 1 / factors_length
    weights = [factor_weight for x in range(factors_length)]

    # Generate Decision Tree
    # print(permutation)
    gen_count += 1
    a = datetime.datetime.now()
    root_node = generate_decision_tree(mcda_type, list(permutation), weights, hotel_name=hotel_name,
                                       rule_year=rule_year, rule_month=rule_month)
    b = datetime.datetime.now()
    c = b - a
    print(str(gen_count) + ": " + str(c) + " - " + str(permutation))
    nodes.append(root_node)

    # Prototype: calculate score on each tree
    # print(RenderTree(root_node, style=AsciiStyle()).by_attr())
    paths = collect_paths(root_node)
    for p_idx, path in enumerate(paths):
        count += 1
        result_name = list(map(lambda k : k.name, path[1:]))
        weight_sum = sum(n.weight for n in path[1:])
        score_sum = sum(n.score for n in path[1:]) if weight_sum == 1 else 0
        result_row = [None] * 7
        result_row[0] = count
        result_row[1:5] = result_name + [None] * (5 - len(result_name))
        result_row[6] = score_sum
        result_score_table.append(result_row)

all cases = 26
1: 0:00:02.882222 - ('stay', 'length_of_stay')
2: 0:00:03.329063 - ('stay', 'country')
3: 0:00:03.231137 - ('stay', 'review_group')
4: 0:00:05.445144 - ('stay', 'room_type')
5: 0:00:04.092102 - ('length_of_stay', 'country')
6: 0:00:03.328740 - ('length_of_stay', 'review_group')
7: 0:00:04.183983 - ('length_of_stay', 'room_type')
8: 0:00:41.465972 - ('country', 'review_group')
9: 0:00:39.540923 - ('country', 'room_type')
10: 0:00:04.653281 - ('review_group', 'room_type')
11: 0:00:05.997696 - ('stay', 'length_of_stay', 'country')
12: 0:00:05.840369 - ('stay', 'length_of_stay', 'review_group')
13: 0:00:07.023380 - ('stay', 'length_of_stay', 'room_type')
14: 0:00:16.041365 - ('stay', 'country', 'review_group')
15: 0:00:14.267378 - ('stay', 'country', 'room_type')
16: 0:00:16.711335 - ('stay', 'review_group', 'room_type')
17: 0:00:07.624089 - ('length_of_stay', 'country', 'review_group')
18: 0:00:06.890000 - ('length_of_stay', 'country', 'room_type')
19: 0:00:09.433867 - ('le

In [9]:
import csv

# print(tabulate(result_score_table, headers='firstrow', tablefmt='fancy_grid'))
with open("experiment-1-combination.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerows(result_score_table)